In [3]:
!python -m pip install seaborn
!python -m pip install scikit-learn

  Using cached seaborn-0.12.2-py3-none-any.whl (293 kB)
  Using cached pandas-2.0.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.3 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 1.2 MB/s eta 0:00:0000:0100:01
  Using cached matplotlib-3.7.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (11.6 MB)
  Using cached contourpy-1.1.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (300 kB)
  Using cached cycler-0.11.0-py3-none-any.whl (6.4 kB)
  Using cached Pillow-9.5.0-cp310-cp310-manylinux_2_28_x86_64.whl (3.4 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 kB 829.5 kB/s eta 0:00:0000:0100:01
  Using cached kiwisolver-1.4.4-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (1.6 MB)
  Using cached fonttools-4.40.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.2 MB)
  Using cached pytz-2023.3-py2.py3-none-any.whl (502 kB)
  Using cached tzdata-2023.3-py2.py3-none-any.whl (341 kB)

[notice] A new rele

In [4]:
!python -m pip install "elasticsearch>=6.4.0,<7.0.0"

  Using cached elasticsearch-6.8.2-py2.py3-none-any.whl (90 kB)
  Using cached urllib3-2.0.3-py3-none-any.whl (123 kB)

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [5]:
from elasticsearch import Elasticsearch
client = Elasticsearch("http://172.17.212.35:9200")
resp = client.search(index="retromotion-indexer_development_products",
                     body={"_source":["descriptions","descriptionsSource","nameSource","shortDescriptionSource","categoriesSource"],
                           "query": {"match_all": {}}})


In [6]:
print("Got %d Hits:" % resp['hits']['total'])

Got 22329 Hits:


In [7]:
import pandas as pd
import re
import numpy as np

import sklearn
import matplotlib.pyplot as plt
import seaborn as sns

In [8]:
df_eng = pd.DataFrame(columns=['name','sdesc','category','catlevel0','catlevel1','catlevel2','catlevel3','catlevel4','catlevel5'])
df_de = pd.DataFrame(columns=['name','sdesc','desc','category','categorypath'])

In [14]:
df_eng.iloc[0]

name         denso dtm82363 thermostat coolant
sdesc                       thermostat coolant
category                            thermostat
catlevel0                            sparepart
catlevel1                       cooling system
catlevel2                           thermostat
catlevel3                                  NaN
catlevel4                                  NaN
catlevel5                                  NaN
Name: 0, dtype: object

In [10]:
def cleanStr(text):
   
    # remove numbers
    # no_number_string = re.sub(r'\d+','',lower_string)

    # remove all punctuation except words and space
    # text = re.sub(r'[^\w\s]','', lower_string)

    # clean = re.compile('<.*?>')

    # text = re.sub(clean,'',text)
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = (re.sub('[\W]+', ' ', text.lower()) + ' ' .join(emoticons).replace('-', ''))

    return text


In [11]:
def readHtml(html):
    dftable = pd.read_html(html)
    return dftable

In [12]:
name_en=[]
name_de=[]

sdesc_en=[]
sdesc_de=[]

desc_en=[]
desc_de=[]

cat_en=[]
cat_de=[]

catpath_en=[]
catpath_de=[]

for hit in resp['hits']['hits']:
    list_row =[]
    for name in hit['_source']['nameSource']:
        if (name["language"]=="en"):
            list_row.append(cleanStr(name["value"]))
        else:
            name_de.append(name["value"])
    
    for sdesc in hit['_source']['shortDescriptionSource']:
        if (sdesc["language"]=="en"):
            list_row.append(cleanStr(sdesc["value"]))
        else:
            sdesc_de.append(sdesc["value"])

    # for desc in hit['_source']['descriptionsSource']:
    #     if (desc["language"]=="en"):
    #         desc_en.append(readHtml(desc["value"]))
    #     else:
    #         desc_de.append(desc["value"])
    
    for cats in hit['_source']['categoriesSource']:
        if (cats["language"]=="en"):
            list_row.append(cleanStr(cats["label"]))
        else:
            desc_de.append(cats["label"])

        if (cats["language"]=="en"):
            categories =cats["path"].split('/')
            i =0
            catls=[]
            for category in categories:

                list_row.append(cleanStr(category))
                i= i+1
            
        else:
            catpath_de.append(cats["path"])
    
    # new_row = pd.DataFrame({'name':_name, 'sdesc':_sdesc, 'category':_cat_en }, index=[0])

    # pd.concat([new_row,df_eng.loc[:]]).reset_index(drop=True)

    for i in range (0,9-len(list_row)):
        list_row.append(np.nan)

    df_eng.loc[len(df_eng)] = list_row



In [ ]:
# df_en = pd.DataFrame({'name':name_en,
#                        'sdesc':sdesc_en,
#                     #    'desc':desc_en,
#                        'category':cat_en,
#                        'categorypath':catpath_en})


# df_de = pd.DataFrame({'name':name_de,
#                        'sdesc':sdesc_de,
#                        'desc':desc_de,
#                        'category':cat_de,
#                        'categorypath':catpath_de})

In [ ]:
print (df_eng.to_latex())

In [ ]:
df_eng.isna().sum()

In [13]:
df_eng.shape

(10, 9)

In [ ]:
df_eng = df_eng.dropna()

In [ ]:
df_eng.shape

In [ ]:
df_eng.head()

In [ ]:
df_eng.describe()

One hot encoding 

In [ ]:
# df_eng = pd.get_dummies(df_eng,columns=["category"])
# df_eng.head()

In [59]:
df_eng.iloc[7]

name         vaico v10 2660 mounting automatic transmission
sdesc                       mounting automatic transmission
category                                           mounting
catlevel0                                         sparepart
catlevel1                                      transmission
catlevel2                            automatic transmission
catlevel3                                          mounting
catlevel4                                               NaN
catlevel5                                               NaN
Name: 7, dtype: object

In [32]:
from sklearn.feature_extraction.text import CountVectorizer
n_gram_vectorizer = CountVectorizer(ngram_range=(1, 1))
n_gram_vectorizer.fit(df_eng['name'] + df_eng['category'] )
transformed_vector_name = n_gram_vectorizer.transform(df_eng['name'])
transformed_vector_sdesc = n_gram_vectorizer.transform(df_eng['sdesc'])
transformed_vector_category = n_gram_vectorizer.transform(df_eng['category'])

# vocabulary


In [33]:
n_gram_vectorizer.vocabulary_

{'denso': 22,
 'dtm82363': 23,
 'thermostat': 33,
 'coolantthermostat': 21,
 'zf': 45,
 '1043': 7,
 '010': 1,
 '297': 11,
 'automatic': 17,
 'transmissionautomatic': 37,
 'transmission': 36,
 '1060': 8,
 '040': 4,
 '003': 0,
 'vaico': 43,
 'v10': 39,
 '0131': 2,
 'camshaftcamshaft': 19,
 'v26': 41,
 '0302': 3,
 'track': 35,
 'control': 20,
 'armtrack': 16,
 'arm': 15,
 'v30': 42,
 '1250': 9,
 'suspension': 31,
 'kitsuspension': 27,
 'kit': 26,
 '0474': 5,
 'tensioner': 32,
 'pulley': 29,
 'timing': 34,
 'belttensioner': 18,
 '2660': 10,
 'mounting': 28,
 'transmissionmounting': 38,
 '4245': 12,
 'shock': 30,
 'absorbershock': 14,
 'absorber': 13,
 'v24': 40,
 '0908': 6,
 'wheel': 44,
 'hubwheel': 25,
 'hub': 24}

In [34]:
transformed_vector_name.shape

(10, 46)

In [35]:
transformed_vector_name.toarray()

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0],
       [0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 1],
       [1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 1],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1,
        0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1,
        0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
        0, 0],
       [0, 0, 0, 0, 0,

In [37]:
n_gram_vectorizer.vocabulary_.get('automatic')

17

In [18]:
vocabulary.items()

NameError: name 'vocabulary' is not defined

In [19]:
from sklearn.model_selection import train_test_split

X_train, x_test, Y_train, y_test = train_test_split(transformed_vector_sdesc,
                                                    transformed_vector_category,
                                                    test_size=0.2,
                                                    random_state=0)

In [20]:
X_train.shape, Y_train.shape

((8, 19), (8, 16))

In [23]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [21]:

Xtrain_ = torch.from_numpy(X_train).float()
Xtest_ = torch.from_numpy(x_test).float()
Xtrain_.shape
Ytrain_ = torch.from_numpy(Y_train).view(1,-1)[0]
Ytest_ = torch.from_numpy(y_test).view(1,-1)[0]
Ytrain_.shape

AttributeError: values not found

In [22]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.fc2 = nn.Linear(hidden_size, hidden_size) 
        self.fc3 = nn.Linear(hidden_size, output_size) 
    
    def forward(self, x):
        x = F.sigmoid(self.fc1(x))
        x = F.sigmoid(self.fc2(x)) 
        x = self.fc3(x)
        
        return F.log_softmax(x, dim=-1)

NameError: name 'nn' is not defined

In [ ]:
model = Net()

In [ ]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

loss_fn = nn.NLLLoss()

In [ ]:
epoch_data = []
epochs = 1001

for epoch in range(1, epochs):

    optimizer.zero_grad()
    Ypred = model(Xtrain_)

    loss = loss_fn(Ypred , Ytrain_)
    loss.backward()

    optimizer.step()
        
    Ypred_test = model(Xtest_)
    loss_test = loss_fn(Ypred_test, Ytest_)
    
    _,pred = Ypred_test.data.max(1)
    
    accuracy = pred.eq(Ytest_.data).sum().item() / y_test.values.size
    epoch_data.append([epoch, loss.data.item(), loss_test.data.item(), accuracy])
    
    if epoch % 100 == 0:
        print ('epoch - %d (%d%%) train loss - %.2f test loss - %.2f accuracy - %.4f'\
               % (epoch, epoch/150 * 10 , loss.data.item(), loss_test.data.item(), accuracy))